In [2]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import yfinance as yf, pandas as pd, shutil, os
import math

df = pd.read_csv('NYSE.csv')

In [66]:
#unusual volume
increased_volume = []

for stock in df['Symbols']:
    stock = stock.upper()
    if '^' in stock:
        pass
    else:
        try:
            stock_info = yf.Ticker(stock)
            hist = stock_info.history(period="5d")
            previous_averaged_volume = hist['Volume'].iloc[1:4:1].mean()
            todays_volume = hist['Volume'][-1]
            previous_close = hist['Close'][-2]
            current_close = hist['Close'][-1]
            if todays_volume > previous_averaged_volume * 4 and previous_close < current_close and todays_volume > 100000:
                print(stock)
                print(previous_averaged_volume)
                print(todays_volume)
                increased_volume.append(stock)
        except:
            pass

print(increased_volume)

0       AACQ
1        AAL
2       AAOI
3       AAON
4       AAPL
        ... 
2198    ZNGA
2199    ZNTL
2200      ZS
2201    ZUMZ
2202    ZYXI
Name: 0, Length: 2203, dtype: object
EDAP
102333.33333333333
1445995
GNTY
15333.333333333334
123106
HGEN
1134300.0
66896576
VTRU
22566.666666666668
1038174208
WEYS
13333.333333333334
3228033603
WILC
6466.666666666667
1041576576
['EDAP', 'GNTY', 'HGEN', 'VTRU', 'WEYS', 'WILC']


In [3]:
buy_list=[]
price_list=[]
stop_loss=[]
s=['EDAP', 'GNTY', 'HGEN', 'VTRU', 'WEYS', 'WILC']

In [4]:
def getinfo(x):
    symbol=x
    state=yf.Ticker(symbol).history(period="max")
    state = state[len(state)-80:len(state)]
    Data=state.loc[:,['Open','High','Low','Close']]
    Data=Data.iloc[::-1]
    return Data

In [5]:
for a in range(len(s)):
    Data=getinfo(s[a])
    
    #settings
    kijun_lookback  = 26
    tenkan_lookback =  9
    chikou_lookback = 26
    senkou_span_projection = 26
    senkou_span_b_lookback = 52

    closing_price=Data.iloc[0,3]
    opening_price=Data.iloc[0,0]
    prev_closing_price=Data.iloc[1,3]
    prev_opening_price=Data.iloc[1,0]
    price_26day=Data.iloc[chikou_lookback-1,1]

    # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    Bhigh=Data[:kijun_lookback].loc[:,['High']].max()
    high=Bhigh[0]
    Blow=Data[:kijun_lookback].loc[:,['Low']].min()
    low=Blow[0]

    kijun_sen=(high+low)/2

    # Kijun-sen (Base Line)prev day: (26-period high + 26-period low)/2))
    high27=Data[1:kijun_lookback+1].loc[:,['High']].max()
    high27=high27[0]
    Blow27=Data[1:kijun_lookback+1].loc[:,['Low']].min()
    low27=Blow27[0]

    kijun_sen_prev=(high27+low27)/2
    # tenkan-sen (Conversion Line):  [(9-period high + 9-period low)/2])
    high=Data[:tenkan_lookback].loc[:,['High']].max()
    high=high[0]
    low=Data[:tenkan_lookback].loc[:,['Low']].min()
    low=low[0]

    tenkan_sen=(high+low)/2

    # tenkan-sen (Conversion Line) prev day:  [(9-period high + 9-period low)/2])
    high27=Data[1:tenkan_lookback+1].loc[:,['High']].max()
    high27=high27[0]
    low27=Data[1:tenkan_lookback+1].loc[:,['Low']].min()
    low27=low27[0]

    tenkan_sen_prev=(high27+low27)/2

    #Chikou span: CLosing price
    chikou=closing_price

    #Senkou span A  [(Conversion Line + Base Line)/2] Current
    Senkou_A=(kijun_sen+tenkan_sen)/2

    #Senkou span A  [(Conversion Line + Base Line)/2] looking at 26 days before
    Bhigh26=Data[senkou_span_projection:kijun_lookback+senkou_span_projection].loc[:,['High']].max()
    high26=Bhigh26[0]
    Blow26=Data[senkou_span_projection:kijun_lookback+senkou_span_projection].loc[:,['Low']].min()
    low26=Blow26[0]
    kijun_sen26=(high26+low26)/2

    chigh26=Data[senkou_span_projection:tenkan_lookback+senkou_span_projection].loc[:,['High']].max()
    high=chigh26[0]
    clow26=Data[senkou_span_projection:tenkan_lookback+senkou_span_projection].loc[:,['Low']].min()
    low=clow26[0]

    tenkan_sen26=(high+low)/2

    Senkou_AP=(kijun_sen26+tenkan_sen26)/2

    #Senkou span B  [(52-period high + 52-period low)/2] Current
    high=Data[:senkou_span_b_lookback].loc[:,['High']].max()
    high=high[0]
    low=Data[:senkou_span_b_lookback].loc[:,['Low']].min()
    low=low[0]

    Senkou_B=(high+low)/2

    #Senkou span B  [(52-period high + 52-period low)/2] Looking at 26 days before
    high26=Data[senkou_span_projection:senkou_span_b_lookback+senkou_span_projection].loc[:,['High']].max()
    high26=high26[0]
    low26=Data[senkou_span_projection:senkou_span_b_lookback+senkou_span_projection].loc[:,['Low']].min()
    low26=low26[0]

    Senkou_BP=(high26+low26)/2
    def momentum_test():
        # first test conversion>base
        if tenkan_sen>kijun_sen and tenkan_sen_prev<kijun_sen_prev:
            print('high momentum breakout!')
            return 1
        elif tenkan_sen>kijun_sen and tenkan_sen_prev>kijun_sen_prev:
            print('momentum')
            return 1
        else:
            print('no breakout')
            return 0
    def lagging_test():
        if chikou>price_26day:
            print('lagging span above past price')
            return 1
        else:
            print('lagging span below price')
            return 0

    n=0
    if opening_price> Senkou_AP and closing_price>Senkou_AP and opening_price> Senkou_BP and closing_price>Senkou_BP and Senkou_A>Senkou_B :
        print('above green cloud')
        n+=momentum_test()
        n+=lagging_test()
    else:
        print('ignore')
    if n==2:
        print(s[a]+',buy')
        buy_list.append(s[a])
        price_list.append(closing_price)
        stop_loss.append(tenkan_sen26)
    
    else:
        print('dont buy')

ignore
dont buy
above green cloud
momentum
lagging span above past price
GNTY,buy
ignore
dont buy
ignore
dont buy
above green cloud
momentum
lagging span above past price
WEYS,buy
above green cloud
momentum
lagging span below price
dont buy


In [11]:
portfolio=float(input('total cash:'))
maxnostocks=10
maxrisk=0.1
todays_cap=portfolio/maxnostocks*len(buy_list)
max_per_stock=maxrisk*portfolio
my_columns = ['Ticker', 'Price', 'Number Of Shares to Buy','Stop Loss']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe['Ticker']=buy_list
final_dataframe['Price']=price_list
final_dataframe['Stop Loss']=stop_loss

total cash:10000


In [14]:
todays_cap

2000.0

In [15]:
#weightage
def weightage(cap,max_per_stock):
    vol_change=[]
    weight=[]
    numberofshares=[]
    for i in range(len(buy_list)):
        symbol=buy_list[i]
        vol=yf.Ticker(symbol).history(period="max")
        vol = vol[len(vol)-26:len(vol)].loc[:,['Volume']]
        vol=vol.iloc[::-1]
        vol=list(vol['Volume'])

        pastav_vol=sum(vol[1:len(vol)])/(len(vol)-1)
        current_vol=vol[0]
        per_change_vol=(current_vol/pastav_vol)*100
        vol_change.append(per_change_vol)
    for u in range(len(buy_list)):
        w=vol_change[u]/sum(vol_change)*cap
        weight.append(w)
    for n in range(len(price_list)):
        if weight[n]>max_per_stock:
            numbershares=math.floor(max_per_stock/price_list[n])
        else:    
            numbershares=math.floor(weight[n]/price_list[n])
        numberofshares.append(numbershares)
    return numberofshares

In [16]:
final_dataframe['Number Of Shares to Buy']=weightage(todays_cap,max_per_stock)

In [17]:
final_dataframe


,Ticker,Price,Number Of Shares to Buy,Stop Loss
0,GNTY,37.509998,26,30.930183
1,WEYS,22.600000,6,18.756139
